reworking of this:

[http://localhost:8888/notebooks/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/e_cloudmle.ipynb#Deploy-model]

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import os
print tf.__version__

1.12.0


In [2]:
PROJECT = 'kouzoh-p-michal'   
BUCKET = 'ml-training-kouzoh-p-michal-appspot-com' 
REGION = 'us-central1' 

MODEL_NAME = 'taxifare'
MODEL_VERSION = 'v1'
TRAINING_DIR = './data/taxi_trained'

In [3]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8' 
os.environ['CLOUDSDK_PYTHON'] = 'python2'
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 

## Training data check
Data is prepared as CSVs and uploaded to the bucket in a separate notebook

In [4]:
%%bash
#gsutil cp gs://$BUCKET/taxifare/ch4/taxi_preproc/test.csv-00000-of-00001 ./data/test.csv
#gsutil cp gs://$BUCKET/taxifare/ch4/taxi_preproc/train.csv-00000-of-00001 ./data/train.csv
#gsutil cp gs://$BUCKET/taxifare/ch4/taxi_preproc/valid.csv-00000-of-00001 ./data/valid.csv

## Remove trained model data

In [16]:
%%bash
rm -rf $TRAINING_DIR/*

## Train model locally

In [19]:
%%bash
# Setup python so it sees the task module which controls the model.py
export PYTHONPATH=${PYTHONPATH}:${PWD}/${MODEL_NAME}
python -m trainer.task \
   --train_data_paths=./data/train.csv \
   --eval_data_paths=./data/valid.csv  \
   --output_dir=${TRAINING_DIR} \
   --train_steps=10000 --job-dir=./data/tmp

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 30, '_num_ps_replicas': 0, '_keep_checkpoint_max': 3, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8b9305f810>, '_model_dir': './data/taxi_trained', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 30, '_num_ps_replicas': 0, '_keep_checkpoint_max': 3, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': T

## Train model locally using gcloud

In [ ]:
%%bash
# Use Cloud Machine Learning Engine to train the model in local file system
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   -- \
   --train_data_paths=${PWD}/taxi-train.csv \
   --eval_data_paths=${PWD}/taxi-valid.csv  \
   --train_steps=1000 \
   --output_dir=${PWD}/${TRAINING_DIR} 

## Local prediction

In [21]:
%%bash
# This model dir is the model exported after training and is used for prediction
#
model_dir=$(ls ${PWD}/${TRAINING_DIR}/export/exporter | tail -1)
# predict using the trained model
gcloud ml-engine local predict  \
    --model-dir=${PWD}/${TRAINING_DIR}/export/exporter/${model_dir} \
    --json-instances=./data/test.json

PREDICTIONS
[7.211677074432373]
[6.5388078689575195]
[7.211677074432373]
[6.5388078689575195]
[6.5388078689575195]
[7.211677074432373]
[9.23028564453125]


## Submit trainig job to cloud

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/ch4/trained
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# Clear the Cloud Storage Bucket used for the training job
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/train*" \
   --eval_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

Check what's in the directory where cloud training outputs the model

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/taxifare/ch4/trained/export/exporter

Deploy model - this just creates a model without any content

In [ ]:
%%bash
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION

 Deploy trained version to the model we created


In [ ]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/taxifare/ch4/trained/export/exporter | tail -1)

echo "MODEL_LOCATION = ${MODEL_LOCATION}"

gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

## Prediction

In [ ]:
%%bash
gcloud ml-engine predict --model=${MODEL_NAME} --version=${MODEL_VERSION} --json-instances=./data/test.json